Running PyTorch on MPS requires MacOS 12.3+ *and* the ARM version of Python installed. We can check the MacOS version with:

In [1]:
import platform; platform.mac_ver()

('12.4', ('', '', ''), 'arm64')

The first item tells us our MacOS version, this must be greater than *12.3* and if it is not then update your Mac!

The final item tells us the OS version our current environment is running in, this should be *arm64*. An alternative check is to print the `platform`:

In [2]:
platform.platform()

'macOS-12.4-arm64-arm-64bit'

If the above displays something like `macOS-12.4-x86_64-i386-64bit` (eg containing `x86`), we have the wrong version of Python installed and must install the correct (ARM) version. If using Anaconda a new ARM environment can be set up like so:

```bash
CONDA_SUBDIR=osx-arm64 conda create -n ml python=3.9 -c conda-forge
```

Here we are setting the conda version variable to use ARM versions for install. We then `create` a new `conda` environment with name (`-n`) `ml`. We use Python `3.9` for this and make sure we have `conda-forge` as a repository in our channels `-c` where these ARM installs can be downloaded from.

Next we activate the environment with `conda activate ml` and modify the `CONDA_SUBDIR` variable to permanently use `osx-arm64`, otherwise this may default back to an incorrect *x84* version during future installs.

```bash
conda env config vars set CONDA_SUBDIR=osx-arm64
```

You may see a message asking you to reactivate the environment for these changes to take effect, if so just switch back to the *base* environment then back into the `ml` environment with:

```bash
conda activate
conda activate ml
```

Now we're ready to install the latest PyTorch version (v1.12 or higher), at the moment this requires that we install the PyTorch nightly preview with:

```bash
pip install -U --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu
```

During downloads you should be able to see something like *Downloading torch-1.1x.x.---**arm64.whl***. That final *arm64.whl* part is important and tells us we are downloading the correct version.

For the examples in this notebook we will also use HF *transformers* and *datasets*.

```bash
pip install transformers datasets
```

---
***Note**: The transformers library uses tokenizers built in Rust (it makes them faster), because we are using this new ARM64 environment we may get **ERROR: Failed building wheel for tokenizers**. If so, we [install Rust](https://huggingface.co/docs/tokenizers/python/v0.9.4/installation/main.html#installation-from-sources) (in the same environment) with:*

```bash
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
```

*And then `pip install transformers datasets` again.*

---

In [3]:
import torch

/opt/homebrew/anaconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First, we check if we can use MPS with:

In [4]:
torch.has_mps

True

Later, we'll need to shift our model and tensors to the MPS device, for that we'll need this `device` variable:

In [5]:
device = torch.device('mps')

Before getting into the model and fine-tuning, we need to download some data to fine-tune our model with...

In [6]:
from datasets import load_dataset

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')
trec

Using custom data configuration default
Reusing dataset trec (/Users/jamesbriggs/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


Dataset({
    features: ['label-coarse', 'label-fine', 'text'],
    num_rows: 1000
})

In [7]:
trec[0]

{'label-coarse': 0,
 'label-fine': 0,
 'text': 'How did serfdom develop in and then leave Russia ?'}

There are also a few data preparation steps we need to perform. We need to tokenize our input text `text`, one-hot encode our labels `label-coarse`, and then place these together in a dataset and dataloader.

For tokenization we need to use a *tokenizer*, we will use the `bert-base-uncased` tokenizer from HF.

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# we have a small dataset so we can tokenize everything at once
# tokenize everything
tokens = tokenizer(
    trec['text'], max_length=512,
    truncation=True, padding='max_length'
)

This returns a list of encoding objects

In [9]:
tokens[:2]

[Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

And we access individual components using (for example):

In [10]:
tokens[0].ids

[101,
 2129,
 2106,
 14262,
 2546,
 9527,
 4503,
 1999,
 1998,
 2059,
 2681,
 3607,
 1029,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

Next we one-hot encode our labels.

In [11]:
import numpy as np

# initialize array to be used
labels = np.zeros(
    (len(trec), max(trec['label-coarse'])+1)
)
# one-hot encode
labels[np.arange(len(trec)), trec['label-coarse']] = 1
labels[:5]

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [12]:
labels = torch.Tensor(labels)

Now we're ready to create the dataset object.

In [13]:
class TrecDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __getitem__(self, idx):
        input_ids = self.tokens[idx].ids
        attention_mask = self.tokens[idx].attention_mask
        labels = self.labels[idx]
        return {
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask),
            'labels': torch.tensor(labels)
        }

    def __len__(self):
        return len(self.labels)

dataset = TrecDataset(tokens, labels)

In [14]:
loader = torch.utils.data.DataLoader(
    dataset, batch_size=1
)

Now let's try training a BERT model, we'll use this and our TREC data to compare inference time on CPU vs MPS.

In [15]:
from transformers import BertForSequenceClassification, BertConfig

config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = max(trec['label-coarse'])+1
model = BertForSequenceClassification(config).to(device)  # remember to move to MPS!

Training prep

In [16]:
# activate training mode of model
model.train()

# initialize adam optimizer with weight decay
optim = torch.optim.Adam(model.parameters(), lr=5e-5)

In [17]:
from time import time
from tqdm.auto import tqdm

loop_time = []
#model.to(device)

# setup loop (using tqdm for the progress bar)
loop = tqdm(loader, leave=True)
for batch in loop:
    batch_mps = {
        'input_ids': batch['input_ids'].to(device),
        'attention_mask': batch['attention_mask'].to(device),
        'labels': batch['labels'].to(device)
    }
    t0 = time()
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # train model on batch and return outputs (incl. loss)
    outputs = model(**batch_mps)
    # extract loss
    loss = outputs[0]
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()
    loop_time.append(time()-t0)
    # print relevant info to progress bar
    loop.set_postfix(loss=loss.item())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1000 [00:00<?, ?it/s]/var/folders/s0/_jtvm_x17pz3grlnzl2j8k6r0000gn/T/ipykernel_2496/3304426640.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(labels)
  6%|▌         | 60/1000 [00:48<12:32,  1.25it/s, loss=0.344]


KeyboardInterrupt: 

In [ ]:
loop_time

---